#Installing and Importing Libraries

In [6]:
!pip install PyPDF2 sentence_transformers groq requests scikit-learn fuzzywuzzy streamlit python-docx docx2txt fpdf reportlab pyngrok -q
!python -m spacy download en_core_web_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 52.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Mounting the Drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Front End

In [12]:
%%writefile app.py

GROQ_API_KEY = "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"

import streamlit as st
from io import BytesIO
import os
import json
import PyPDF2
import spacy
import random
import matplotlib.pyplot as plt
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
from groq import Groq
from google.colab import drive
import nltk
from typing import List, Dict, Optional
from collections import deque
from PyPDF2 import PdfReader
from nltk.tokenize import sent_tokenize
nltk.download('punkt', quiet=True)

JOB_ROLES = [
    "Actuarial Analyst",
    "Computer Vision Engineer",
    "Data Analyst",
    "Data Engineer",
    "Data Scientist",
    "Financial Market Analyst",
    "Junior Analyst Developer",
    "MERN Developer",
    "Product Associate",
    "SDE_ SWE_ Software Development Engineer_ Software Engineer"
]

RAG_FOLDER_PATH = "/content/drive/My Drive/LLM_RESUME_DATA/LLM Project Final/Mid Review 2/RAG folder - skills of particular roles/"

def extract_text_from_pdf(pdf_path):
    job_description = ""
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page in pdf_reader.pages:
            job_description += page.extract_text()
    return job_description

def find_best_matching_role(sentence_model, job_description):
    job_description_embedding = sentence_model.encode(job_description, convert_to_tensor=True)
    job_roles_embeddings = sentence_model.encode(JOB_ROLES, convert_to_tensor=True)
    cosine_similarities = util.pytorch_cos_sim(job_description_embedding, job_roles_embeddings)
    best_match_index = cosine_similarities.argmax().item()
    return JOB_ROLES[best_match_index], cosine_similarities[0][best_match_index].item()

def ask_question(question, modelname):
    client = Groq(api_key=GROQ_API_KEY)
    completion = client.chat.completions.create(
        model=modelname,
        messages=[
            {"role": "system", "content": "You are an expert in understanding and generating Job Descriptions and Resumes in CSE field."},
            {"role": "user", "content": question}
        ],
        temperature=0.5,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    response = ""
    for chunk in completion:
        response += chunk.choices[0].delta.content or ""
    return response

def analyze_job_description(pdf_path, output_path, modelname):
    try:
        sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
        nlp = spacy.load("en_core_web_sm")
        job_description = extract_text_from_pdf(pdf_path)
        best_role, _ = find_best_matching_role(sentence_model, job_description)
        get_skills_prompt = f"Here is a job description: {job_description}.\n\nUnderstand it and list all the required skills in a Python list format. Respond only with a Python list of skill names, and avoid any extra text or new lines."
        get_eligibility_prompt = f"Here is a job description: {job_description}.\n\nUnderstand it and list all the required eligibility criteria in a Python list format. Respond only with a Python list of eligibility items, and avoid any extra text or new lines."
        skills = ask_question(get_skills_prompt, modelname)
        eligibility = ask_question(get_eligibility_prompt, modelname)
        result = {
            "role": best_role,
            "skills": skills,
            "eligibility": eligibility
        }
        with open(output_path, 'w') as json_file:
            json.dump(result, json_file, indent=4)
        return result
    except Exception as e:
        import traceback
        traceback.print_exc()
        return None

def analyze_resume(pdf_path, output_path, modelname):
    try:
        resume_text = extract_text_from_pdf(pdf_path)
        person_details_prompt = (
            f"Extract the person's details (name, phone, email, location) from this resume text: {resume_text}. "
            "Return the details strictly as a Python dictionary with keys 'name', 'phone', 'email', and 'location', and provide no extra text."
        )
        education_prompt = (
            f"Extract the education details (degree, university, dates) from this resume text: {resume_text}. "
            "Return the information strictly as a list of dictionaries, where each dictionary contains 'degree', 'university', and 'dates'. Provide no extra text."
        )
        skills_prompt = (
            f"Extract the skills from this resume text: {resume_text}. "
            "Return the skills strictly as a Python list of skill names, and provide no extra text."
        )
        work_ex_prompt = (
            f"Extract the work experience (company, role, dates, responsibilities) from this resume text: {resume_text}. "
            "Return the information strictly as a list of dictionaries, with each dictionary containing 'company', 'role', 'dates', and 'responsibilities'. Provide no extra text."
        )
        projects_prompt = (
            f"Extract the projects (name, description, technologies used) from this resume text: {resume_text}. "
            "Return each project strictly as a dictionary with 'name', 'description', and 'technologies_used' keys, and return all projects in a list format. Provide no extra text."
        )
        extras_prompt = (
            f"Extract any additional information (awards, certifications, volunteer experience) from this resume text: {resume_text}. "
            "Return the information strictly as a list of dictionaries with each dictionary containing 'type' and 'details' for each extra item. Provide no extra text."
        )
        person_details = ask_question(person_details_prompt, modelname)
        education = ask_question(education_prompt, modelname)
        skills = ask_question(skills_prompt, modelname)
        work_ex = ask_question(work_ex_prompt, modelname)
        projects = ask_question(projects_prompt, modelname)
        extras = ask_question(extras_prompt, modelname)
        result = {
            "person_details": person_details,
            "education": education,
            "skills": skills,
            "work_ex": work_ex,
            "projects": projects,
            "extras": extras
        }
        with open(output_path, 'w') as json_file:
            json.dump(result, json_file, indent=4)
        return result
    except Exception as e:
        print(f"Error processing file: {str(e)}")
        return None

def enhance_resume(jd_json_path, resume_json_path, output_path, modelname):
    try:
        with open(jd_json_path, 'r') as f:
            jd_data = json.load(f)
        with open(resume_json_path, 'r') as f:
            resume_data = json.load(f)

        skills = resume_data.get('skills', {})
        work_ex = resume_data.get('work_ex', {})
        projects = resume_data.get('projects', {})
        extras = resume_data.get('extras', {})

        skills_prompt = f"""
        Original skills section: {skills}
        Job description: {jd_data}

        Please enhance the skills section by:
        1. Aligning it with the job description
        2. Using industry-standard terminology and formatting
        3. Prioritizing the most relevant skills first

        Enhanced skills section (maintain the original format):
        """

        work_ex_prompt = f"""
        Original work experience: {work_ex}
        Job description requirements: {jd_data}

        Please enhance the work experience section by:
        1. Highlighting experiences that match the job requirements
        2. Using industry-standard terminology
        3. Quantifying achievements where possible
        4. Focusing on relevant responsibilities

        Enhanced work experience section (maintain the original format):
        """

        projects_prompt = f"""
        Original projects section: {projects}
        Job description requirements: {jd_data}

        Please enhance the projects section by:
        1. Highlighting projects that demonstrate required skills
        2. Using industry-standard technical terminology
        3. Emphasizing problem-solving and results
        4. Including relevant technical details

        Enhanced projects section (maintain the original format):
        """

        extras_prompt = f"""
        Original extras section: {extras}
        Job requirements: {jd_data}

        Please enhance the extras section by:
        1. Highlighting relevant certifications and achievements
        2. Including industry-specific accomplishments
        3. Adding relevant professional memberships or activities
        4. Removing less relevant information

        Enhanced extras section (maintain the original format):
        """

        enhanced_skills = ask_question(skills_prompt, modelname)
        enhanced_work_ex = ask_question(work_ex_prompt, modelname)
        enhanced_projects = ask_question(projects_prompt, modelname)
        enhanced_extras = ask_question(extras_prompt, modelname)

        enhanced_resume = {
            "person_details": resume_data.get('person_details', {}),
            "education": resume_data.get('education', {}),
            "skills": enhanced_skills,
            "work_ex": enhanced_work_ex,
            "projects": enhanced_projects,
            "extras": enhanced_extras
        }

        with open(output_path, 'w') as f:
            json.dump(enhanced_resume, f, indent=4)

        return enhanced_resume

    except FileNotFoundError as e:
        print(f"Error: Could not find input file - {str(e)}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format - {str(e)}")
        return None
    except Exception as e:
        print(f"Error enhancing resume: {str(e)}")
        return None

class RAGSystem:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.client = Groq(api_key=api_key)
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.conversation_history = deque(maxlen=5)

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        try:
            with open(pdf_path, 'rb') as file:
                reader = PdfReader(file)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() + " "
            return text.strip()
        except Exception as e:
            print(f"Error reading PDF {pdf_path}: {e}")
            return ""

    def create_chunks(self, text: str, chunk_size: int = 500) -> List[str]:
        sentences = sent_tokenize(text)
        chunks = []
        current_chunk = ""

        for sentence in sentences:
            if len(current_chunk) + len(sentence) < chunk_size:
                current_chunk += " " + sentence
            else:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = sentence

        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks

    def get_embeddings(self, texts: List[str]):
        return self.embedding_model.encode(texts)

    def find_relevant_chunks(self, query: str, chunks: List[str], top_k: int = 3) -> List[str]:
        query_embedding = self.embedding_model.encode([query])
        chunk_embeddings = self.embedding_model.encode(chunks)
        similarities = chunk_embeddings @ query_embedding.T
        top_indices = similarities.argsort(axis=0)[-top_k:][::-1]
        return [chunks[idx[0]] for idx in top_indices]

    def get_llm_response(self, query: str, context: str) -> str:
        history_context = "\n".join([f"Q: {q}\nA: {a}" for q, a in self.conversation_history])

        prompt = f"""Based on the following context and conversation history, please answer the question.
        If the answer cannot be found in the context, say so clearly.

        Context: {context}

        Previous Conversation:
        {history_context}

        Question: {query}

        Answer:"""

        try:
            completion = self.client.chat.completions.create(
                model="llama-3.1-70b-versatile",
                messages=[
                    {"role": "system", "content": "You are an expert in analyzing job descriptions and career-related documents."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.5,
                max_tokens=1024,
                stream=True
            )

            response = ""
            for chunk in completion:
                if chunk.choices[0].delta.content:
                    response += chunk.choices[0].delta.content

            return response.strip()

        except Exception as e:
            print(f"Error getting LLM response: {e}")
            return "I encountered an error while generating the response."

    def update_conversation_history(self, query: str, answer: str):
        self.conversation_history.append((query, answer))

    def process_query(self, folder_path: str, query: str) -> Dict[str, any]:
        try:
            pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
            if not pdf_files:
                return {"answer": "No PDF files found in the specified folder.", "sources": []}
            all_chunks = []
            sources_map = {}

            for pdf_file in pdf_files:
                file_path = os.path.join(folder_path, pdf_file)
                text = self.extract_text_from_pdf(file_path)
                if text:
                    chunks = self.create_chunks(text)
                    all_chunks.extend(chunks)
                    for chunk in chunks:
                        sources_map[chunk] = pdf_file

            if not all_chunks:
                return {"answer": "No content could be extracted from the PDFs.", "sources": []}

            relevant_chunks = self.find_relevant_chunks(query, all_chunks)
            sources = [sources_map[chunk] for chunk in relevant_chunks]
            context = "\n".join(relevant_chunks)
            answer = self.get_llm_response(query, context)
            self.update_conversation_history(query, answer)

            return {
                "answer": answer,
                "sources": list(set(sources))
            }

        except Exception as e:
            print(f"Error in RAG pipeline: {e}")
            return {
                "answer": "An error occurred while processing your query.",
                "sources": []
            }

def enhance_resume_with_rag(jd_json_path: str, resume_json_path: str, output_path: str, rag_folder_path: str, modelname: str) -> Dict:
    try:
        rag_system = RAGSystem(api_key=GROQ_API_KEY)

        with open(jd_json_path, 'r') as f:
            jd_data = json.load(f)
        with open(resume_json_path, 'r') as f:
            resume_data = json.load(f)

        role_query = f"What are the key skills, responsibilities, and best practices for a {jd_data.get('role', '')} role?"
        rag_knowledge = rag_system.process_query(rag_folder_path, role_query)

        skills = resume_data.get('skills', {})
        work_ex = resume_data.get('work_ex', {})
        projects = resume_data.get('projects', {})
        extras = resume_data.get('extras', {})

        skills_prompt = f"""
        Original skills section: {skills}
        Job description requirements: {jd_data}
        Industry knowledge from our database: {rag_knowledge['answer']}

        Please enhance the skills section by:
        1. Aligning it with the job description
        2. Incorporating relevant industry-standard skills from our knowledge base
        3. Using proper terminology and formatting
        4. Prioritizing the most relevant skills first

        Enhanced skills section (maintain the original format):
        """

        work_ex_prompt = f"""
        Original work experience: {work_ex}
        Job description requirements: {jd_data}
        Industry knowledge: {rag_knowledge['answer']}

        Please enhance the work experience section by:
        1. Highlighting experiences that match the job requirements
        2. Using industry-standard terminology
        3. Quantifying achievements where possible
        4. Focusing on relevant responsibilities

        Enhanced work experience section (maintain the original format):
        """

        projects_prompt = f"""
        Original projects section: {projects}
        Job description requirements: {jd_data}
        Industry knowledge: {rag_knowledge['answer']}

        Please enhance the projects section by:
        1. Highlighting projects that demonstrate required skills
        2. Using industry-standard technical terminology
        3. Emphasizing problem-solving and results
        4. Including relevant technical details

        Enhanced projects section (maintain the original format):
        """

        extras_prompt = f"""
        Original extras section: {extras}
        Job requirements: {jd_data}
        Industry knowledge: {rag_knowledge['answer']}

        Please enhance the extras section by:
        1. Highlighting relevant certifications and achievements
        2. Including industry-specific accomplishments
        3. Adding relevant professional memberships or activities
        4. Removing less relevant information

        Enhanced extras section (maintain the original format):
        """

        enhanced_skills = ask_question(skills_prompt, modelname)
        enhanced_work_ex = ask_question(work_ex_prompt, modelname)
        enhanced_projects = ask_question(projects_prompt, modelname)
        enhanced_extras = ask_question(extras_prompt, modelname)

        enhanced_resume = {
            "person_details": resume_data.get('person_details', {}),
            "education": resume_data.get('education', {}),
            "skills": enhanced_skills,
            "work_ex": enhanced_work_ex,
            "projects": enhanced_projects,
            "extras": enhanced_extras,
            "rag_sources": rag_knowledge.get('sources', [])
        }

        with open(output_path, 'w') as f:
            json.dump(enhanced_resume, f, indent=4)

        return enhanced_resume

    except FileNotFoundError as e:
        print(f"Error: Could not find input file - {str(e)}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format - {str(e)}")
        return None
    except Exception as e:
        print(f"Error enhancing resume: {str(e)}")
        return None

def read_file(file):
    pdf_reader = PdfReader(file)
    text = []
    for page in pdf_reader.pages:
        page_text = page.extract_text()
        if page_text:
            text.extend(page_text.splitlines())
    return "\n".join(text)

def get_pdfs(file1, file2):
    resume_text = read_file(file1)
    jd_text = read_file(file2)
    resume_pdf = BytesIO(resume_text.encode("utf-8"))
    jd_pdf = BytesIO(jd_text.encode("utf-8"))
    return resume_pdf, jd_pdf

def best_resume(jd_json_path, resume_json_path, enhanced_resume_path, best_model, best_version):
    if best_model == "llama-3.1-70b-versatile" and best_version == "R1":
        return enhance_resume(jd_json_path, resume_json_path, enhanced_resume_path, best_model)
    elif best_model == "llama-3.1-70b-versatile" and best_version == "R2":
        return enhance_resume_with_rag(jd_json_path, resume_json_path, enhanced_resume_path, RAG_FOLDER_PATH, best_model)
    elif best_model == "llama-3.2-90b-text-preview" and best_version == "R1":
        return enhance_resume(jd_json_path, resume_json_path, enhanced_resume_path, best_model)
    elif best_model == "llama-3.2-90b-text-preview" and best_version == "R2":
        return enhance_resume_with_rag(jd_json_path, resume_json_path, enhanced_resume_path, RAG_FOLDER_PATH, best_model)
    else:
        return resume_json_path

def process_files(file1, file2, best_model, best_version):
    import tempfile
    import os
    from reportlab.pdfgen import canvas
    from reportlab.lib.pagesizes import letter
    from reportlab.pdfbase import pdfmetrics
    from reportlab.pdfbase.ttfonts import TTFont
    from reportlab.lib import colors

    with tempfile.TemporaryDirectory() as temp_dir:
        try:
            log_file_path = os.path.join(temp_dir, "processing_log.txt")
            with open(log_file_path, "w") as log_file:
                log_file.write("Starting process...\n")

            resume_pdf_path = os.path.join(temp_dir, "resume.pdf")
            jd_pdf_path = os.path.join(temp_dir, "jd.pdf")
            jd_json_path = os.path.join(temp_dir, "jd.json")
            resume_json_path = os.path.join(temp_dir, "resume.json")
            enhanced_resume_path = os.path.join(temp_dir, "enhanced_resume.json")

            try:
                with open(resume_pdf_path, "wb") as f:
                    f.write(file1.getvalue())
                with open(jd_pdf_path, "wb") as f:
                    f.write(file2.getvalue())

                with open(log_file_path, "a") as log_file:
                    log_file.write("PDFs saved successfully.\n")
            except Exception as e:
                with open(log_file_path, "a") as log_file:
                    log_file.write(f"Error saving PDFs: {str(e)}\n")
                return None

            try:
                jd = analyze_job_description(jd_pdf_path, jd_json_path, best_model)
                resume = analyze_resume(resume_pdf_path, resume_json_path, best_model)

                if not jd or not resume:
                    with open(log_file_path, "a") as log_file:
                        log_file.write("Error analyzing documents.\n")
                    return None

                with open(log_file_path, "a") as log_file:
                    log_file.write("Documents analyzed successfully.\n")
            except Exception as e:
                with open(log_file_path, "a") as log_file:
                    log_file.write(f"Error during analysis: {str(e)}\n")
                return None

            try:
                enhanced_data = best_resume(jd_json_path, resume_json_path, enhanced_resume_path, best_model, best_version)
                if not enhanced_data:
                    with open(log_file_path, "a") as log_file:
                        log_file.write("Error enhancing resume.\n")
                    return None
            except Exception as e:
                with open(log_file_path, "a") as log_file:
                    log_file.write(f"Error during enhancement: {str(e)}\n")
                return None

            try:
                output = BytesIO()
                c = canvas.Canvas(output, pagesize=letter)
                width, height = letter

                enhanced_text = json.dumps(enhanced_data, indent=2)
                lines = enhanced_text.split('\n')

                y = height - 50
                line_height = 12
                left_margin = 50

                def new_page():
                    nonlocal y
                    c.showPage()
                    c.setFont("Helvetica", 10)
                    y = height - 50

                c.setFont("Helvetica", 10)

                for line in lines:
                    if y < 50:
                        new_page()


                    while len(line) > 100:
                        split_point = line[:100].rfind(' ')
                        if split_point == -1:
                            split_point = 100

                        c.drawString(left_margin, y, line[:split_point])
                        line = line[split_point:].lstrip()
                        y -= line_height

                        if y < 50:
                            new_page()

                    c.drawString(left_margin, y, line)
                    y -= line_height

                c.save()

                with open(log_file_path, "a") as log_file:
                    log_file.write("Enhanced resume created successfully.\n")

                output.seek(0)
                return output.getvalue()

            except Exception as e:
                with open(log_file_path, "a") as log_file:
                    log_file.write(f"Error creating PDF: {str(e)}\n")
                return None

        except Exception as e:
            try:
                with open(log_file_path, "a") as log_file:
                    log_file.write(f"An error occurred: {str(e)}\n")
            except:
                pass
            return None

best_model_and_r_path = "/content/drive/My Drive/LLM_RESUME_DATA/LLM Project Final/Mid Review 2/best_model_and_r.txt"
with open(best_model_and_r_path, "r") as f:
    lines = f.readlines()
    best_model = lines[0].strip()
    best_version = lines[1].strip()

st.title("ResumeTune")
file1 = st.file_uploader("Upload Resume", type=["pdf"])
file2 = st.file_uploader("Upload Job Description", type=["pdf"])

if st.button("Enhance your Resume"):
    if file1 is not None and file2 is not None:
        with st.spinner("Enhancing your resume... This may take a few minutes."):
            merged_pdf_data = process_files(file1, file2, best_model, best_version)
            if merged_pdf_data:
                st.success("Enhancement complete!")
                st.download_button(
                    label="Download Enhanced Resume",
                    data=merged_pdf_data,
                    file_name="enhanced_resume.pdf",
                    mime="application/pdf",
                )
            else:
                st.error("Failed to enhance resume. Please try again.")
    else:
        st.error("Please upload both the resume and job description.")

Overwriting app.py


In [ ]:
from pyngrok import ngrok, conf
import os
import time
import subprocess
import signal
import psutil

def kill_process_on_port(port):
    try:
        for proc in psutil.process_iter(['pid', 'name', 'connections']):
            try:
                for conn in proc.connections():
                    if conn.laddr.port == port:
                        os.kill(proc.pid, signal.SIGTERM)
                        time.sleep(1)
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                continue
    except Exception as e:
        print(f"Error killing process on port {port}: {e}")

def start_streamlit_server(port):
    try:
        kill_process_on_port(port)

        os.environ['STREAMLIT_SERVER_ADDRESS'] = '0.0.0.0'
        os.environ['STREAMLIT_SERVER_PORT'] = str(port)

        process = subprocess.Popen(
            ["streamlit", "run", "app.py",
             "--server.address", "0.0.0.0",
             "--server.port", str(port)],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )

        time.sleep(10)
        return process
    except Exception as e:
        print(f"Error starting Streamlit: {e}")
        return None

def setup_ngrok(port, auth_token):
    try:
        # Configure ngrok
        conf.get_default().auth_token = auth_token
        conf.get_default().region = 'in'

        # Kill any existing ngrok processes
        ngrok.kill()

        # Start ngrok tunnel - simplified configuration
        public_url = ngrok.connect(port)

        return public_url
    except Exception as e:
        print(f"Error setting up ngrok: {e}")
        return None

def main():
    PORT = 8501
    AUTH_TOKEN = "2oA2P8ECRxHzy0VQamBIrcr6w04_5nvMNVm9QnXijSFTaqxoq"

    try:
        print("Starting Streamlit server...")
        streamlit_process = start_streamlit_server(PORT)

        if not streamlit_process:
            raise Exception("Failed to start Streamlit server")

        print("Setting up ngrok tunnel...")
        public_url = setup_ngrok(PORT, AUTH_TOKEN)

        if not public_url:
            raise Exception("Failed to establish ngrok tunnel")

        print("\n" + "="*50)
        print(f"Streamlit App URL: {public_url}")
        print("="*50 + "\n")

        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            print("Shutting down...")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        print("Cleaning up...")
        ngrok.kill()
        if 'streamlit_process' in locals():
            streamlit_process.terminate()
        kill_process_on_port(PORT)

if __name__ == "__main__":
    main()

Starting Streamlit server...
Setting up ngrok tunnel...

Streamlit App URL: NgrokTunnel: "https://1516-34-145-198-186.ngrok-free.app" -> "http://localhost:8501"

